In [ ]:
def solar_info_annuel(coord_string):

    coord_string = coord_string.strip('()')  # Retire les parenthèses si présentes
    lat, lon = map(float, coord_string.split(','))

    # Délai de 1-2 secondes entre chaque requête
    time.sleep(1.5)
    
    #lon, lat = colonne
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)

    # Paramètres à renseigner
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": f"2024-01-01",
        "end_date": f"2024-12-31",
        "daily": ["sunshine_duration", "temperature_2m_mean"],
        "hourly": "shortwave_radiation",
        "timezone": "Europe/Berlin"
    }
    responses = openmeteo.weather_api(url, params=params)

    response = responses[0]
    

    # Process hourly data
    hourly = response.Hourly()
    hourly_shortwave_radiation = hourly.Variables(0).ValuesAsNumpy()

    hourly_data = {"date": pd.date_range(
        start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
        end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = hourly.Interval()),
        inclusive = "left"
    )}

    hourly_data["shortwave_radiation"] = hourly_shortwave_radiation

    hourly_dataframe = pd.DataFrame(data = hourly_data)


    # Nettoyage pour exploitation des données du df Hourly
    hourly_dataframe['month'] = hourly_dataframe['date'].dt.month_name(locale='English')
    hourly_dataframe['month_num'] = hourly_dataframe['date'].dt.month

    hourly_dataframe = hourly_dataframe.iloc[2:]

    df_ghi = hourly_dataframe.groupby(['month', 'month_num']).agg(
        ghi_mean_W_m2=('shortwave_radiation', 'mean')
    )

    df_ghi = df_ghi.sort_values(by='month_num')


    # Process daily data
    daily = response.Daily()
    daily_sunshine_duration = daily.Variables(0).ValuesAsNumpy()
    daily_temperature_2m_mean = daily.Variables(1).ValuesAsNumpy()

    daily_data = {"date": pd.date_range(
        start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
        end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = daily.Interval()),
        inclusive = "left"
    )}

    daily_data["sunshine_duration"] = daily_sunshine_duration
    daily_data["temperature_2m_mean"] = daily_temperature_2m_mean

    daily_dataframe = pd.DataFrame(data = daily_data)


    # Nettoyage pour exploitation des données du df daily
    daily_dataframe['sunshine_duration_hours'] = daily_dataframe['sunshine_duration'] / 3600
    daily_dataframe.drop(columns='sunshine_duration', inplace=True)

    daily_dataframe = daily_dataframe.iloc[1:]

    daily_dataframe['month'] = daily_dataframe['date'].dt.month_name(locale='English')
    daily_dataframe['month_num'] = daily_dataframe['date'].dt.month
    daily_dataframe['days_in_month'] = daily_dataframe['date'].dt.daysinmonth



    df_weather = daily_dataframe.groupby(['month', 'month_num']).agg(
        temp_mean_c=('temperature_2m_mean', 'mean'),
        sunshine_duration_mean_hour=('sunshine_duration_hours', 'mean'),
        days_in_month=('days_in_month', 'first')  
    )


    df_weather = df_weather.sort_values(by='month_num')

    # Concaténation des dfs

    df = pd.merge(df_weather, df_ghi, on='month')
    

    # ==================#
    #     Panneaux       #
    #  Photovoltaïques  #
    # ==================#

    # --- Paramètres spécifiques à l'installation ---
    # Ajout de paramètres types, servant d'exemples, 
    EFFICIENCY_PANEL = 0.20  # Exemple : rendement de 20 % (par ex. pour un panneau de 400W, diviser par sa surface en m²)

    ANGLE_ORIENTATION_FACTOR = 1.00 

    TEMP_COEFFICIENT_PMAX = 0.004  # Typique : 0.004 soit 0,4 % de perte de puissance par °C 

    SYSTEM_LOSS_FACTOR = 0.85      # Tient compte des pertes du système : onduleur, câblage, saleté, ombrage, etc. (typiquement entre 0.75 et 0.90)

    DELTA_TEMP_PANEL = 20          # Écart estimé entre la température du panneau et la température ambiante (en °C)

    # =================#
    #     Formules     #
    # =================#

    # 1. Calcul de l’irradiance journalière en kWh/m²/jour
    # Ce calcul suppose que ghi_mean_W_m2 et sunshine_duration_mean_hour représentent des moyennes journalières pour le mois.
    df['ghi_journalier_kWh_m2_jour'] = (df['ghi_mean_W_m2'] * df['sunshine_duration_mean_hour']) / 1000

    # 2. Estimation de la température de fonctionnement des panneaux
    # La température du panneau est généralement supérieure à la température ambiante
    df['panel_temp_c'] = df['temp_mean_c'] + DELTA_TEMP_PANEL

    # 3. Calcul du facteur de correction de température
    # Les performances diminuent lorsque la température dépasse 25°C
    df['correction_temp'] = 1 - (df['panel_temp_c'] - 25) * TEMP_COEFFICIENT_PMAX

    # On s’assure que correction_temp reste dans des bornes raisonnables (évite des gains irréalistes ou pertes trop fortes)
    df['correction_temp'] = df['correction_temp'].clip(lower=0.95, upper=1.05)

    # 4. Calcul de la production d’énergie journalière par m² (kWh/jour/m²)
    # On combine tous les facteurs :
    # GHI_journalier * Rendement * Facteur d’orientation * Correction température * Pertes système
    df['energie_jour_kWh/j/m2'] = df['ghi_journalier_kWh_m2_jour'] * \
                                EFFICIENCY_PANEL * \
                                ANGLE_ORIENTATION_FACTOR * \
                                df['correction_temp'] * \
                                SYSTEM_LOSS_FACTOR

    # Arrondi à 2 décimales pour l’affichage
    df['energie_jour_kWh/j/m2'] = round(df['energie_jour_kWh/j/m2'], 2)

    # 5. Calcul de la production mensuelle d’énergie par m² (kWh/mois/m²)
    # On multiplie la production journalière par le nombre de jours dans le mois
    df['energie_mois_kWh/mois/m2'] = round(df['energie_jour_kWh/j/m2'] * df['days_in_month'], 2)

    return round(df['energie_mois_kWh/mois/m2'].sum(), 2)